# Installing PLINK
Make sure you have PLINK on your computer and ready to use. An easy installation method is via Anaconda (v1.90b4). To do so, enter `conda install -c bioconda plink` into your command line. Alternatively, please follow the instructions on the [PLINK website](https://www.cog-genomics.org/plink2). To check that PLINK has been installed, enter `plink` in your command line. <br>

# What is PLINK? 
PLINK is a popular genome analysis toolset with numerous functionalities. PLINK formatted genotype files can come in a binary fileset comprised of bed/bim/fam files, or a standard text fileset comprised of ped/map files. You can read about these file formats [here](https://www.cog-genomics.org/plink2/formats#bed). Some basic functions of PLINK which can be useful for parsing/QCing large genetic datasets include functions for selecting SNPs on specific chromosomes (`--chr`); passing a list of variants or individuals to keep or exclude (`--keep`, `--exclude`); pruning for linkage disequilibrum (`--indep`); compute relationship matrices (`--make-grm-bin`); performing population stratification (`--pca/--cluster`); and merging multiple genetic filesets (`--merge-list`) - to just scratch the surface of its functionalities!

# Data download from 1000 Genomes phase 3
We will be working with data from the 1000 Genomes phase 3 reference genomes. Due to the large datasets, we will not be working with whole genome data. Instead, we will just work with data from chromosome 1 to perform population stratification. You will see that this is more than enough genetic data for our purposes!

## Reported population data

The 2504 samples in the phase3 release are from 26 populations which can be categorised into five super-populations 
by continent (listed below).  As well as the global AF in the INFO field. We added AF for each super-population to the INFO field. <br>

| Super-population | Code |
| - | - |
| East Asian | EAS |
| South Asian | SAS |
| African | AFR |
| European | EUR |
| Ad Mixed American | AMR |

A full table including the 26 populations can be found [here](http://www.internationalgenome.org/category/population/). <br>

The population assignments for each individual can be found in `integrated_call_samples_v3.20130502.ALL.panel`


In [ ]:
!wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel

## Download HapMap Phase 3 genotype data for chr1 (.vcf format)
`wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz`

## Convert .vcf to PLINK binary fileset (.bed/.bim/.fam)
`plink --vcf ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz --allow-no-samples --out ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.sites`


# Randomly extract variants from chr1 with $p=0.01$

Because these reference genotypes are so comprehensive, the data is very large and needs to be thinned for feasible computational time for the purposes of this workshop.

`plink --bfile ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.sites --thin 0.01 --make-bed --out ALL.chr1.phase3.1`

# Population stratification with PCA

__Principal component analysis (PCA)__ is a common method for population stratification for genotype data. Because ancestry is a confounding variable and there can be nuances in the genetics across populations, genetic studies often focus on specific populations/subpopulations. <br>

### What is PCA?
What do you do when you have an overwhelming number of variables to consider for an analysis (here, each individual allele corresponds to? You might try to focus on just a few key variables. This is analagous to reducing the dimensions of your feature space, in technical terms - which is exactly what PCA does! There are a variety of methods for dimensionality reduction, but the two main classes are *feature elimination* and *feature extraction*, and many methods can be used to achieve these goals. Without going into the details, it's a lot of linear algebra. If you'd like to learn more, here are some good resources:<br>
http://setosa.io/ev/principal-component-analysis/ <br>
https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c <br>

We will be using PLINK's `--pca` routine, implemented as follows: <br>

`--pca {count} <header> <tabs> <var-wts>` <br>

By default, `--pca` extracts the top 20 principal components of the variance-standardized relationship matrix; you can change the number by passing a numeric parameter. Eigenvectors are written to plink.eigenvec, and top eigenvalues are written to plink.eigenval. The 'header' modifier adds a header line to the .eigenvec file(s), and the 'tabs' modifier makes the .eigenvec file(s) tab- instead of space-delimited.

In [ ]:
!plink --bfile ALL.chr1.phase3.1 --pca header tabs --out ALL.chr1.phase3.1

# Visualize!
Let's see how well PLINK was able to stratify the 1000 Genomes super populations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import numpy as np

In [ ]:
# load eigenvalues
eigenvals = open('ALL.chr1.phase3.1.eigenval').read().splitlines()
eigenvals = [float(x) for x in eigenvals]
print(eigenvals[:5])

In [ ]:
# load eigenvecs
eigenvecs = pd.read_table('ALL.chr1.phase3.1.eigenvec')
eigenvecs.head()

In [ ]:
eigenvecs.shape

In [ ]:
# Plot first two principal components
x = eigenvecs['PC1']
y = eigenvecs['PC2']
plt.scatter(x,y,alpha=0.3)

In [ ]:
# load pop data
pop = pd.read_table('integrated_call_samples_v3.20130502.ALL.panel', usecols=[0,1,2,3])
pop.head()

In [ ]:
# plot first two PCs
fig = plt.figure()
ax = fig.add_subplot(111)

# generate colormap for super pops
colors=cm.rainbow(np.linspace(0,1,len(set(pop['super_pop']))))

# plot (PC1,PC2) for each individual in each super pop
for p, col in zip(set(pop['super_pop']),colors):
    inds = pop[pop['super_pop']==p]['sample']
    pc1 = eigenvecs[eigenvecs['FID'].isin(inds)]['PC1']
    pc2 = eigenvecs[eigenvecs['FID'].isin(inds)]['PC2']
    ax.scatter(pc1,pc2, c = col, alpha=0.3, label = p)
    
ax.legend()

You can see that the five populations cluster neatly. There is some overlap between EUR (Europeans) and AMR (Ad Mixed American). This is not unexpected for mixed ancestry individuals from America. There is also a sort of merge betwen Africans AFR (Africans) and AMR. This is also not unexpected, and is indicative of mixed ancestry as well. <br>

More interestingly, the SAS (South Asian) super population completely overlaps the AMR super population. Let's a take a closer look at what populations comprise these super populations.

In [ ]:
sas_pops = pop[pop['super_pop']=='SAS']['pop']
amr_pops = pop[pop['super_pop']=='AMR']['pop']
print('SAS populations: %s \nAMR populations: %s' % (', '.join(set(sas_pops)), ', '.join(set(amr_pops))))


| Population code | Population | Super-population code|
| - | - | - |
| ITU | Indian Telugu from the UK | SAS |
| BEB | Bengali from Bangladesh | SAS |
| PJL | Punjabi from Lahore, Pakistan	 | SAS |
| STU | Sri Lankan Tamil from the UK | SAS |
| GIH | Gujarati Indian from Houston, Texas	| SAS |
| MXL | Mexican Ancestry from Los Angeles, USA | AMR |
| PEL | Peruvians from Lima, Peru | AMR | 
| CLM | Colombians from Medellin, Colombia | AMR |
| PUR | Puerto Ricans from Puerto Rico | AMR |

What do you think accounts for this large overlap? <br>